# RAG


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import bitsandbytes
import accelerate

device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

In [3]:
device

'cuda:0'

In [4]:
!nvidia-smi

Wed Feb 21 15:56:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A4000               On  | 00000000:00:05.0  On |                  Off |
| 41%   39C    P8              16W / 140W |    318MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
from datasets import load_dataset, Dataset

path = '/home/paperspace/certibot/data/certibot-data.jsonl'

raw_dataset = load_dataset('json', data_files=path, split='train')
raw_dataset

Using custom data configuration default-3748c19a797a83a1
Reusing dataset json (/home/paperspace/.cache/huggingface/datasets/json/default-3748c19a797a83a1/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


Dataset({
    features: ['topic', 'article'],
    num_rows: 97
})

In [7]:
data = raw_dataset.to_pandas()

data = data.reset_index()
data['context'] = data['topic'] + ': ' + data['article']
data['index'] = data['index'].astype((str))

dataset = Dataset.from_pandas(data)

In [8]:
# checkpoint = "Cohere/Cohere-embed-multilingual-v3.0"
checkpoint = "intfloat/multilingual-e5-large-instruct"
embed_dim = 1024

In [9]:
embedding = HuggingFaceEmbeddings(model_name=checkpoint)

You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.3.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [10]:
import pinecone

api_key = 'a344a187-8b52-422d-97c2-96628ef67ef6'

pc = pinecone.Pinecone(api_key=api_key)

In [11]:
index_name = 'certibot-rag'

In [12]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=embed_dim,
        metric='cosine',
        spec=pinecone.PodSpec('gcp-starter')
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.00097,
 'namespaces': {'': {'vector_count': 97}},
 'total_vector_count': 97}

In [13]:
batch_size = 4

In [14]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(0, len(data), batch_size))

for i in range(0, len(data), batch_size):
  end = min(len(data), i + batch_size)
  batch = data[i:end]

  ids = batch['index']
  texts = batch['context']

  embeds = embedding.embed_documents(texts)

  metadata = [{
      'context': x['context'],
      'topic': x['topic']
      } for _, x in batch.iterrows()]

  vectors = list(zip(ids, embeds, metadata))
  index.upsert(vectors=vectors)
  progress_bar.update(1)

  0%|          | 0/25 [00:00<?, ?it/s]

# RAG functions

In [15]:
def retrieve(query:str, top_k:int=5) -> list:
  """
  Embeds a user query, retrieves top_k relevant contexts and returns them for
  use by the LLM.
  """

  query_embed = embedding.embed_query(query)
  res = index.query(vector=query_embed, top_k=top_k, include_metadata=True)
  contexts = [x['metadata']['context'] for x in res['matches']]

  return contexts

In [16]:
res = retrieve("comment beneficier de l'offre free")

In [17]:
res

["Offre Free Mobile: Si vous êtes abonné mobile Forfait Free ou Série Free, vous bénéficiez de tarifs exclusifs sur CertiDeal. Pour profiter de l'offre, il suffit de vous connecter sur votre espace Abonné mobile Free, rubrique Mon compte -> Mon avantage.\nSi vous n'êtes pas encore abonné mobile Free, vous pouvez aussi bénéficier de l'offre exclusive CertiDeal avec Free Pour cela :\nChoisissez votre smartphone incluant les 5 % de remise minimum sur abonnés.certideal.com et valider votre panier sur CertiDeal. Après le paiement, vous serez redirigé automatiquement sur le site mobile Free pour souscrire votre forfait.",
 "Puis-je bénéficier de la remise Free tout au long de l'année?: En souscrivant un forfait mobile Free, vous pourrez bénéficier de la remise de 5% minimum sur l'achat de votre smartphone CertiDeal à tout moment de l'année, dans la limite d'un smartphone par an et par forfait.",
 "Est-ce que je dois acheter un téléphone CertiDeal pour revendre mon ancien appareil ?: Notre of

In [24]:
from transformers import MixtralForCausalLM, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# import bitsandbytes

# model_name = "mistralai/Mistral-7B-v0.1"
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"


tokenizer = AutoTokenizer.from_pretrained(model_name)


# model = AutoModelForCausalLM.from_pretrained(model_name,
#                                             #  quantization_config=bnb_config)


model = MixtralForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map='auto',
    load_in_8bit=False,
    load_in_4bit=True,
    )

# bnb_config = BitsAndBytesConfig(load_in_4bit=True,
#                                 bnb_4bit_quant_type='nf4',
#                                 bnb_4bit_use_double_quant=True,
#                                 bnb_4bit_compute_dtype=torch.bfloat16)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [38]:
torch.cuda.empty_cache()

In [45]:
def rag(query:str, contexts:list) -> str:
  print('> RAG Called')

  context = ""
  for i, text in enumerate(contexts):
    context += f'*{i}: {text}\n\n'

  prompt = f"""
  Tu es un assistant pour les clients de CertiDeal. CertiDeal vends des smartphones reconditionnés.
  Ci-dessous est une requête d'un utilisateur ainsi que quelques contextes pertinents.
  Réponds à la question en fonction des informations contenues dans ces contexts.
  Les réponses doivent être précises et relativement courtes.
  Si tu ne trouve pas la réponse à la question, dis "Je ne sais pas".

  Contextes:
  {context}

  Question: {query}
  Crée une réponse informative sans recopier la requête. Montre que tu as compris la demande en apportant une réponse précise et pertinente.
  """

  inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
  input_ids = inputs['input_ids'].to(device)
  attention_mask = inputs['attention_mask'].to(device)
  # input_ids = inputs['input_ids'].to(device).half().int()
  # attention_mask = inputs['attention_mask'].to(device).half().int()
  outputs = model.generate(
      input_ids,
      attention_mask=attention_mask,
      temperature=0.0,
      max_new_tokens=256,
      )
  answer = tokenizer.decode(outputs[0].to(device), skip_special_tokens=True)

  return answer

In [46]:
out = rag(query="comment souscrire à l'offre free", contexts=res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


> RAG Called


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 3 has a total capacity of 15.53 GiB of which 198.94 MiB is free. Including non-PyTorch memory, this process has 15.31 GiB memory in use. Of the allocated memory 15.05 GiB is allocated by PyTorch, and 66.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [31]:
!nvidia-smi

Wed Feb 21 16:11:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


|   0  NVIDIA RTX A4000               On  | 00000000:00:05.0  On |                  Off |
| 41%   40C    P5              33W / 140W |  15222MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A4000               On  | 00000000:00:07.0 Off |                  Off |
| 41%   47C    P8              36W / 140W |  13640MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
|   2  NVIDIA RTX A4000               On  | 00000000:00:09.0 Off |                  Off |
| 41%   39C    P8              35W / 140W |  13628MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+---------

In [ ]:
out

'<s> \n  Tu es un assistant pour les clients de CertiDeal. CertiDeal vends des smartphones reconditionnés.\n  Tu dois répondre à toutes les questions concernant CertiDeal, ses produits, ses services et ses partenariats.\n\n  Ci-dessous est une requête d\'un utilisateur ainsi que quelques contextes pertinents.\n  Réponds à la question en fonction des informations contenues dans ces contexts.\n  Si tu ne trouve pas la réponse à la question, dis "Je ne sais pas".\n\n  Contextes:\n  Offre Free Mobile: Si vous êtes abonné mobile Forfait Free ou Série Free, vous bénéficiez de tarifs exclusifs sur CertiDeal. Pour profiter de l\'offre, il suffit de vous connecter sur votre espace Abonné mobile Free, rubrique Mon compte -> Mon avantage.\nSi vous n\'êtes pas encore abonné mobile Free, vous pouvez aussi bénéficier de l\'offre exclusive CertiDeal avec Free Pour cela :\nChoisissez votre smartphone incluant les 5 % de remise minimum sur abonnés.certideal.com et valider votre panier sur CertiDeal. Ap

In [ ]:
tokenizer.decode(token_ids=out[0].squeeze().to(device))

'<s> \n  Tu es un assistant pour les clients de CertiDeal. CertiDeal vends des smartphones reconditionnés.\n  Tu dois répondre à toutes les questions concernant CertiDeal, ses produits, ses services et ses partenariats.\n\n  Ci-dessous est une requête d\'un utilisateur ainsi que quelques contextes pertinents.\n  Réponds à la question en fonction des informations contenues dans ces contexts.\n  Si tu ne trouve pas la réponse à la question, dis "Je ne sais pas".\n\n  Contextes:\n  Offre Free Mobile: Si vous êtes abonné mobile Forfait Free ou Série Free, vous bénéficiez de tarifs exclusifs sur CertiDeal. Pour profiter de l\'offre, il suffit de vous connecter sur votre espace Abonné mobile Free, rubrique Mon compte -> Mon avantage.\nSi vous n\'êtes pas encore abonné mobile Free, vous pouvez aussi bénéficier de l\'offre exclusive CertiDeal avec Free Pour cela :\nChoisissez votre smartphone incluant les 5 % de remise minimum sur abonnés.certideal.com et valider votre panier sur CertiDeal. Ap